In [1]:
import pandas as pd
import jaydebeapi
import io as io

def handle_excel():
    global sf
    global dfn
    global dfh
    global regon_code
    # 读取标准项目信息,files 提供美奥提供excel文件路径和名称，newitem新项目的sheet名称，hisitem 老项目的sheet名称
    # 对新项目和老项目的一级分类、二级分类和收费项目名称分别进行编码，完成后保存留底
    files="D:/Works/ProjectDoc/09 上海M/04 构建/集团诊疗项目标准化高值成本匹配到物料编码-叶军2022.5.6(1).xlsx"
    newitem="新项目高值耗材匹配-无锡"
    hisitem="老项目高值耗材匹配-无锡"
    #读取数据
    dfn = pd.DataFrame(
    pd.read_excel(
        files,
        sheet_name=newitem,header=0,usecols=[0,1,2,6]))
    dfh = pd.DataFrame(
    pd.read_excel(
        files,
        sheet_name=hisitem,header=0,usecols=[0,1,10,6]))
    dfb = pd.DataFrame(
    pd.read_excel('D:/C/工作簿1.xlsx',
                  sheet_name='Sheet6',
                  header=0,
                  usecols=None,
                  dtype={2: str}))
    regon_code = "3202"
    dfn.insert(0, "regon_code", regon_code)

    dfh.insert(0, "regon_code", regon_code)
    dfh.rename(columns={"老项目":"治疗项目名称"},inplace=True)
    dfh.insert(3,"治疗项目名称",dfh.pop("治疗项目名称"))

    dfb['大类编码'] = regon_code + dfb['大类编码']
    dfb['小类编码'] = regon_code + dfb['小类编码']

    # 这里已经merge好了
    # 新项目
    dfn = pd.merge(dfn, dfb, how='inner').rename(columns={
    '大类编码': '项目大类编码',
    '小类编码': '项目小类编码'
})
    dfn.insert(1, '项目大类编码', dfn.pop('项目大类编码'))
    dfn.insert(3, '项目小类编码', dfn.pop('项目小类编码'))
    
    # 老项目
    dfh = pd.merge(dfh, dfb, how='inner').rename(columns={
    '大类编码': '项目大类编码',
    '小类编码': '项目小类编码'
})
    dfh.insert(1, '项目大类编码', dfh.pop('项目大类编码'))
    dfh.insert(3, '项目小类编码', dfh.pop('项目小类编码'))
# map 用法
# map需要对映射表做去重处理
# dfn.insert(0, '项目大类编码', dfn['项目大类'].map(dfb.set_index('项目大类')['大类编码'].drop_duplicates()))
# dfn.insert(2, '项目小类编码', dfn['项目小类'].map(dfb.set_index('项目小类')['小类编码'].drop_duplicates()))

    sf=dfn.drop_duplicates(subset=["项目大类",  "治疗项目名称"]).sort_values(by=["项目大类", "项目小类",  "治疗项目名称"],axis=0)
    sfh=dfh.drop_duplicates(subset=["项目大类", "治疗项目名称"]).sort_values(by=["项目大类", "项目小类",  "治疗项目名称"],axis=0)

#分类排序，把序号形成字符方法2 lambda表达式
#sf['治疗项目编码'] =sf['治疗项目名称'].groupby(sf['项目大类']).rank().apply(lambda x:str(x).split(".")[0].zfill(4))
#分类排序，把序号形成字符方法1 浮点转成整数
    sf.insert(5 ,'治疗项目编码', sf["项目大类编码"]+'-N'+sf['治疗项目名称'].groupby(sf['项目大类']).rank().astype(int).astype(str).str.zfill(4))
    sf.pop("EAS对应物料编码")
    sfh.insert(5 ,'治疗项目编码', sfh["项目大类编码"]+'-H'+sfh['治疗项目名称'].groupby(sfh['项目大类']).rank().astype(int).astype(str).str.zfill(4))
    sfh.pop("EAS对应物料编码")
     
    dfn.insert(5, '治疗项目编码', dfn['治疗项目名称'].map(sf.set_index('治疗项目名称')['治疗项目编码'].drop_duplicates()))
    dfh.insert(5, '治疗项目编码', dfh['治疗项目名称'].map(sfh.set_index('治疗项目名称')['治疗项目编码'].drop_duplicates()))

    sf=pd.concat([sf,sfh],axis=0).drop_duplicates(subset=["治疗项目名称"]).reset_index(drop=True).sort_values("治疗项目编码")
    dfn=pd.concat([dfn,dfh],axis=0).drop_duplicates(subset=["治疗项目名称","EAS对应物料编码"]).reset_index(drop=True).sort_values("治疗项目编码")
# 保存处理完成的数据
    path = f'D:/C/{regon_code}高值收费项目导入.xlsx'
    dfnsheet = f"{regon_code}高值收费项目对应物料整理表"
    sfsheet= f"{regon_code}高值收费项目导入表"
    with pd.ExcelWriter(path,mode='w') as writer:
        dfn.to_excel(writer,
                 sheet_name=dfnsheet,
                 index=False,
                 columns=[
                     "regon_code", "项目大类编码", "项目大类", "项目小类编码", "项目小类","治疗项目编码",
                     "治疗项目名称","EAS对应物料编码"
                 ])
        sf.to_excel(writer,
                 sheet_name=sfsheet,
                 index=False,
                 columns=[
                     "regon_code", "项目大类编码", "项目大类", "项目小类编码", "项目小类","治疗项目编码",
                     "治疗项目名称"
                 ])
        
    sf=sf.rename(columns={
    'regon_code': 'FREGIONCODE',
    '项目大类编码':'FCATEGORYNUMBER',
    '项目大类':'FCATEGORYNAME',
    '项目小类编码': 'FCATEGORYSUBCODE',
    '项目小类':'FCATEGORYSUBNAME',
    '治疗项目编码':'FPAYITEMNUMBER',
    '治疗项目名称': 'FPAYITEMNAME'
    }).sort_values(
    by=["FCATEGORYNUMBER", "FPAYITEMNUMBER"], axis=0)
    
    dfn.drop(["regon_code",'项目大类编码','项目大类','项目小类编码','项目小类'],axis=1,inplace = True)
    dfn=dfn.rename(columns={'治疗项目编码':'FITEM_NUMBER','治疗项目名称': 'FITEM_NAME','EAS对应物料编码':'FMATERIAL_CODE'}).sort_values(by=['FITEM_NUMBER'],axis=0)
    
# 定义数据库连接
def db_conn():
    url = 'jdbc:oracle:thin:@8.142.180.78:1521:MAEASDB'
    driver = 'oracle.jdbc.driver.OracleDriver'
    jarFile = 'D:/Tools/JDBC/ojdbc8.jar'
    userTo = 'MAEAS'
    passwordTo = 'MAEAS'
    global connTo
    connTo = jaydebeapi.connect(jclassname=driver,
                          url=url,
                          driver_args=[userTo, passwordTo],
                          jars=jarFile)
    
# # 定义把handle_excel函数处理的结果插入到数据库中
# def handle_insertdb(name):
#     # 把excel处理完成的数据存入数据表后进行
#     #尝试把dataframe转成tuples元组类型，实验executemany begin
#     # name='CT_CUS_PAYITEMSCATE_INIT'
#     columns = sf.columns if name =='CT_CUS_PAYITEMSCATE_INIT' else dfn.columns
#      # 获取要出入数据的列名称       
#     tmp = ''
#     fields = ''
#     length_columns = len(columns)
#     for i in range(length_columns): # 获取要插入的列的数量及插入代码
#       tmp = tmp + ':' + str(i+1)
#       fields = fields + columns[i]
#       if i < length_columns-1:
#             tmp += ','
#             fields += ','
#                 # 要执行插入操作的SQL
#     insert_sql = f'''insert into {name} ({fields}) values({tmp}) '''
#     print('datafram insert',insert_sql)

#     insert_data = []
#     # for rows in sf.itertuples(): # 遍历要插入的数据DataFrame
#     #         row = rows[1:] # 获取需要插入的数据。注意：rows[0]是索引。
#     #         insert_data.append(row)
#     # if (rows[0]+1) % 100 == 0 or rows[0] == len(sf)+1: # 每次插入100条。跟上面代码不同的是，我们这里直接使用索引来判断每次插入的数量和是否达到最后一行数据。
#     if name =='CT_CUS_PAYITEMSCATE_INIT':
#         for rows in sf.itertuples(): # 遍历要插入的数据DataFrame
#             row = rows[1:] # 获取需要插入的数据。注意：rows[0]是索引。
#             insert_data.append(row)
#             if (rows[0]+1) % 100 == 0 or rows[0] == len(sf)+1: # 每次插入100条。跟上面代码不同的是，我们这里直接使用索引来判断每次插入的数量和是否达到最后一行数据。
#                 try: 
#                     print("在执行把收费项目excel导入数据库中...")
#                     cursTo = connTo.cursor()
#                     cursTo.executemany(insert_sql,insert_data)
#                     insert_data = []
#                     print(name,"收费项目整理表插入执行成功\n")
#                     handle_categoryid()
#                 except Exception as e: 
#             #     #sql_exceptions.append((beginrow,endrow, e))
#                     print('收费项目整理表插入执行过程有问题:',e)
#                 finally:
#                     cursTo.close()
#     else:
#         for rows in dfn.itertuples(): # 遍历要插入的数据DataFrame        
#             row = rows[1:] # 获取需要插入的数据。注意：rows[0]是索引。
#             insert_data.append(row)
#             if (rows[0]+1) % 100 == 0 or rows[0] == len(dfn)+1: # 每次插入100条。跟上面代码不同的是，我们这里直接使用索引来判断每次插入的数量和是否达到最后一行数据。
#                 try: 
#                     print("在执行把收费项目excel导入数据库中...")
#                     cursTo = connTo.cursor()
#                     cursTo.executemany(insert_sql,insert_data)
#                     insert_data = []
#                     print(name,"收费项目整理表插入执行成功\n")
#                 except Exception as e: 
#             #     #sql_exceptions.append((beginrow,endrow, e))
#                     print('收费项目整理表插入执行过程有问题:',e)
#                 finally:
#                     cursTo.close()
    
# # #尝试把ddataframe转成tuples元组类型，实验executemany end
# def handle_categoryid():
#     cursTo = connTo.cursor()
#     # 一级分类
#     sqlStr ="insert into ct_cus_payitemscate(FID, FNUMBER, FNAME_L2, CFREGIONID, FCREATORID, FCREATETIME, FLASTUPDATEUSERID,FLASTUPDATETIME, FCONTROLUNITID) select newbosid('F699FE2C'),FCATEGORYNUMBER,FCATEGORYNAME,ctrl.fid,'00000000-0000-0000-0000-000000000000CCE7AED4',sysdate,'00000000-0000-0000-0000-000000000000CCE7AED4',sysdate,'00000000-0000-0000-0000-000000000000CCE7AED4'"
#     sqlStr = sqlStr+" from (select distinct FCATEGORYNUMBER, FCATEGORYNAME, FREGIONCODE from ct_cus_payitemscate_init) s left join t_org_ctrlunit ctrl on s.FREGIONCODE = ctrl.FNUMBER where FREGIONCODE =" "'%s'" %(regon_code) + "  order by FCATEGORYNUMBER"
#     try:  
#         print("在执行一级分类插入中...")
#         cursTo.execute(sqlStr)
#         print(regon_code,"一级分类插入成功\n")
#     except Exception as e: 
#         print('一级分类插入执行过程有问题:',e)

#     sqlStr=''
#     sqlStr ="update ct_cus_payitemscate_init t set FCATEGORYID=(select fid from ct_cus_payitemscate s where t.FCATEGORYNUMBER=s.FNUMBER) where t.FCATEGORYID is null"
#     try: 
#         print("在更新辅助表的一级分类ID中...")
#         cursTo.execute(sqlStr)
#         print(regon_code,"更新辅助表的一级分类ID成功\n")
#     except Exception as e: 
#         print('更新辅助表的一级分类ID执行过程有问题:',e)

# # 3.1 插入二级分类
#     sqlStr ="insert into ct_cus_payitemscatesub(FID,FNUMBER, FNAME_L2, CFREGIONID, CFCATEGORYID, FCREATORID, FCREATETIME, FLASTUPDATEUSERID, FLASTUPDATETIME, FCONTROLUNITID)   select newbosid('4A528874'), FCATEGORYSUBCODE,FCATEGORYSUBNAME,ctrl.fid,FCATEGORYID,'00000000-0000-0000-0000-000000000000CCE7AED4',sysdate,'00000000-0000-0000-0000-000000000000CCE7AED4',sysdate,'00000000-0000-0000-0000-000000000000CCE7AED4'"
#     sqlStr =sqlStr+" from (select distinct FCATEGORYNUMBER, FCATEGORYNAME, FREGIONCODE, FCATEGORYSUBCODE, FCATEGORYSUBNAME, FCATEGORYID from ct_cus_payitemscate_init) s left join t_org_ctrlunit ctrl on s.FREGIONCODE = ctrl.FNUMBER where FREGIONCODE =" "'%s'" %(regon_code) + " order by FCATEGORYNUMBER"
#     try: 
#         print("在执行二级分类插入中...")
#         cursTo.execute(sqlStr)
#         print(regon_code,"二级分类插入成功\n")
#         sqlStr ="" 
#     except Exception as e: 
#         print('二级分类插入执行过程有问题:',e)

# # 3.2 更新辅助表的二级分类ID
#     sqlStr =""
#     sqlStr ="update ct_cus_payitemscate_init t set FCATEGORYSUBID=(select fid from ct_cus_payitemscatesub s where t.FCATEGORYSUBCODE=s.FNUMBER) where t.FCATEGORYSUBID is null"
#     try: 
#         print("在执行更新辅助表的二级分类ID中...")
#         cursTo.execute(sqlStr)
#         print(regon_code,"更新辅助表的二级分类ID成功\n")
#     except Exception as e: 
#         print('更新辅助表的二级分类ID执行过程有问题:',e)

# # ************四、根据辅助表形成最终的收费项目*************/
# # 4.1、收费项目插入
#     sqlStr =""
#     sqlStr ="insert into ct_cus_payitems(FID, FNUMBER, FCREATORID, FCREATETIME, FLASTUPDATEUSERID, FLASTUPDATETIME, FCONTROLUNITID, CFCATESUBID, FNAME_L2, CFUSESTATUS, CFISLOWVAL, CFREGIONID)  select newbosid('C6A0F53D'),FPAYITEMNUMBER,'00000000-0000-0000-0000-000000000000CCE7AED4', sysdate, '00000000-0000-0000-0000-000000000000CCE7AED4', sysdate,'00000000-0000-0000-0000-000000000000CCE7AED4',FCATEGORYSUBID,FPAYITEMNAME,'0',FPAYITETYPE,ctrl.fid"
#     sqlStr = sqlStr+" from  ct_cus_payitemscate_init s left join t_org_ctrlunit ctrl on s.FREGIONCODE = ctrl.FNUMBER where FREGIONCODE=" "'%s'" %(regon_code) + " order by FCATEGORYNUMBER"
#     try: 
#         print("在执行收费项目插入中...")
#         cursTo.execute(sqlStr)
#         print(regon_code,"收费项目插入成功\n")
#     except Exception as e: 
#         print('收费项目插入执行过程有问题:',e)
#     finally:
#         cursTo.close()
    
# ************五、形成收费项目和物料的导入表*************/    
if __name__ == '__main__':
    handle_excel()
#   db_conn()
#   handle_insertdb('CT_MAPPING')
#   handle_categoryid()